In [1]:
%matplotlib inline
import h5py
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Input
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception  # TensorFlow ONLY
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

import warnings
warnings.filterwarnings('ignore')

In [2]:
models_filename = '/Users/Barbara/Desktop/Ironhack/Final_Project/food-101/v8-vgg16-model-1/v8_vgg16_model_1.h5'
image_dir = '/Users/Barbara/Desktop/Ironhack/Final_Project/food-101/food-101/images'
image_size = (224, 224)
batch_size = 16
epochs = 80

In [3]:
# 5gb of images won't fit in my memory. use datagenerator to go across all images.
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = False,
fill_mode = "nearest",
zoom_range = 0,
width_shift_range = 0,
height_shift_range=0,
rotation_range=0)

train_generator = train_datagen.flow_from_directory(
image_dir,
target_size = (image_size[0], image_size[1]),
batch_size = batch_size, 
class_mode = "categorical")

num_of_classes = len(train_generator.class_indices)

Found 101000 images belonging to 101 classes.


In [4]:
model = VGG16(weights=None, include_top=False, input_shape=(image_size[0], image_size[1], 3))

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(101*2, activation="relu")(x)
x = Dense(101*2, activation="relu")(x)
predictions = Dense(101, activation="softmax")(x)
model_final = Model(input=model.input, output=predictions)
model_final.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
model_final.load_weights(models_filename)

2024-07-08 19:03:03.954266: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-07-08 19:03:03.954313: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-07-08 19:03:03.954325: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-07-08 19:03:03.954599: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-08 19:03:03.954641: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Input
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import warnings

warnings.filterwarnings('ignore')

# Ensure TensorFlow uses the GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# File paths and parameters
models_filename = '/Users/Barbara/Desktop/Ironhack/Final_Project/food-101/v8-vgg16-model-1/v8_vgg16_model_1.keras'
image_dir = '/Users/Barbara/Desktop/Ironhack/Final_Project/food-101/food-101/images'
image_size = (224, 224)
batch_size = 16
epochs = 80

# Data augmentation and data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    fill_mode="nearest",
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=30
)

train_generator = train_datagen.flow_from_directory(
    image_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical"
)

num_of_classes = len(train_generator.class_indices)

# Load the base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

# Add custom layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dense(512, activation="relu")(x)
predictions = Dense(num_of_classes, activation="softmax")(x)

# Create the final model
model_final = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model_final.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

# Load weights if available
if os.path.exists(models_filename):
    model_final.load_weights(models_filename)

# Callbacks
checkpoint = ModelCheckpoint("best_model.keras", monitor='loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='loss', patience=10, verbose=1, mode='min')
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)

callbacks = [checkpoint, early_stopping, tensorboard]

# Train the model
model_final.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    callbacks=callbacks
)

Num GPUs Available:  1
Found 101000 images belonging to 101 classes.
Epoch 1/80


2024-07-09 09:13:38.534312: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-07-09 09:13:52.755348: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 3 of 8
2024-07-09 09:14:10.056315: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 7 of 8
2024-07-09 09:14:13.611211: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


2433/6312 ━━━━━━━━━━━━━━━━━━━━ 4:51:07 5s/step - accuracy: 0.0096 - loss: 4.6928

2024-07-09 12:16:48.399239: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: TimeoutError: [Errno 60] Operation timed out
Traceback (most recent call last):

  File "/Applications/Utilities/anaconda3/envs/tensorflow_macos/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/Applications/Utilities/anaconda3/envs/tensorflow_macos/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Applications/Utilities/anaconda3/envs/tensorflow_macos/lib/python3.9/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Applications/Utilities/anaconda3/envs/tensorflow_macos/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py", line 260, in _get_iterator
    for i, batch in enumerate(gen_fn()):

  File "/Applicati

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) UNKNOWN:  TimeoutError: [Errno 60] Operation timed out
Traceback (most recent call last):

  File "/Applications/Utilities/anaconda3/envs/tensorflow_macos/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/Applications/Utilities/anaconda3/envs/tensorflow_macos/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Applications/Utilities/anaconda3/envs/tensorflow_macos/lib/python3.9/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Applications/Utilities/anaconda3/envs/tensorflow_macos/lib/p

In [ ]:
preds = model_final.evaluate_generator(train_generator, steps=800, workers=8, use_multiprocessing=True)
preds

In [ ]:
# routine for human evaluation - use the generator so we can see how well it can predict
for n in range(100):
    _ = train_generator.next()
    image, classifier = (_[0][0],_[1][0]) # take the first image from the batch
    index = np.argmax(classifier)
    answer = list(train_generator.class_indices.keys())[index]
    predicted = model_final.predict(np.asarray([image]))
    predicted_answer_index = np.argmax(predicted[0])
    predicted_answer = list(train_generator.class_indices.keys())[predicted_answer_index]

    plt.imshow(image)
    plt.show()

    print('correct answer is: ', answer)
    print()
    print('CNN thinks it''s:', predicted_answer)